In [5]:
# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash

# Configure the necessary Python module imports
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

shelter = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
#app = dash. Dash(__name__)  #Need to uncomment to run dash app

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,# not allow user to edit table/data
        style_table={'height': '100%', 'overflowY': 'auto'}, # Adjust table height
        page_size=10, #set rows per page
        sort_mode="multi", #allow sorting multiple column at once
        sort_action="native", # Enable column sorting
        row_selectable="single", # Enable single-row selection
        selected_rows=[0], # Initialize selected row (first row)
        filter_action="native", # Enable column filtering
        page_action="native", # Enable pagination for data table, splitting data into multiple pages
        page_current=0, # Set initial page display to first page       
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])
#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
 dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
 if index is None:
   row = 0
 else: 
   row = index[0]
# Austin TX is at [30.75,-97.48]
 return [
    dl.Map(style={'width': '1000px', 'height': '500px'},
       center=[30.75,-97.48], zoom=10, children=[
       dl.TileLayer(id="base-layer-id"),
       # Marker with tool tip and popup
       # Column 14 and 15 define the grid-coordinates for 
       # the map
       # Column 5 defines the breed for the animal
       # Column 10 defines the name of the animal
       dl.Marker(position=[dff.iloc[row,14],dff.iloc[row,15]],
          children=[
          dl.Tooltip(dff.iloc[row,5]),
          dl.Popup([
             html.H1("Animal Name"),
            html.P(dff.iloc[row,10])
         ])
      ])
   ])
]
#Run App
#if __name__ == '__main__': app. run_server(debug=True)    #may need to run dash app for geomap if jupyter not able to display map via JupyterDash

app

Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', '0'],
      dtype='object')
Dash is running on http://127.0.0.1:10279/

Dash is running on http://127.0.0.1:10279/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 98] Address already in use